## Task 1 [3p]

Implement simplified word2vec with negative sampling from scratch (using pure numpy). Assume that in the training data objects and contexts are given explicitly, one pair per line, and objects are on the left; see the [data file](https://drive.google.com/file/d/1WoBT5OrTlNnjHg6jN90RnRb7krpEzFsP/view?usp=drive_link). The result of the training should be object vectors. Please, write them to a file using *natural* text format, ie

<pre>
word1 x1_1 x1_2 ... x1_N 
word2 x2_1 x2_2 ... x2_N
...
wordK xK_1 xK_2 ... xk_N
</pre>

Use the loss with negative sampling (NS) as in [Mikolov et al. 2013](https://arxiv.org/pdf/1310.4546) (see section 2.2). The loss function is as follows:  
Given:  
- A **center word** w_c
A **true context word** w_o
k **negative samples**: words not in the context (denoted as w_1, ..., w_k)
Word vectors:
- v_c = embedding of the center word (from input matrix)
- u_o = embedding of the context word (from output matrix)
- u_i = embeddings of negative samples

$$
 L = -\log \sigma(u_o^T v_c) - \sum_{i=1}^k \log \sigma(-u_i^T v_c);
$$

(see [SKOS info](https://skos.ii.uni.wroc.pl/course/view.php?id=738#section-10) for more details)


Compute the gradient manually. You can use some gradient clipping, or regularizaton.


**Remark**: the data is specially prepared to make the learning process easier. 
Present vectors using the code below. In this task we define success as 'obtaining a result which looks definitely not random'


u - object vector, v - context vector  
o - object, c - conteext   
$P(o|c) = \frac{\exp(u_o^T v_c)}{\sum_{w\in V} \exp(u_w^T v_c)}$

**Negative Sampling:**  
$$J_{neg-sample}(u_o, v_c, u_i) = -\log \sigma(u_o^T v_c) - \sum_{k \in \{\text{K sampled indices}\}} \log \sigma(-u_i^T v_c)$$


In [1]:
import numpy as np
from collections import Counter
from tqdm import tqdm

In [3]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [ ]:
embedding_dim = 100 # 100-300
epochs = 3   # 3-6
learning_rate = 0.003
lr_decay = 0.0001  # linear lr decay
neg_samples = 5

In [ ]:
with open('task1_objects_contexts_polish.txt', 'r') as f:
    pairs = [line.strip().split() for line in f if line.strip()]

# use part of data for training
N_pairs = len(pairs)
pairs = np.random.permutation(pairs)
pairs = pairs[:int(0.01 * N_pairs)]
print(f"Using {len(pairs)} pairs for training.")

# generate vocab of all words, separated when they are center and context
objects = set([obj for obj, ctx in pairs])
contexts = set([ctx for obj, ctx in pairs])
print(f"{len(objects)} unique objects, {len(contexts)} unique contexts")

# index words
object_idx = {w: i for i, w in enumerate(objects)}
idx_object = {i: w for i, w in enumerate(objects)}

context_idx = {w: i for i, w in enumerate(contexts)}
idx_context = {i: w for i, w in enumerate(contexts)}


# create vector embedings
embed_center = [[0]*embedding_dim]*len(objects) # v_w when w is center
embed_context = [[0]*embedding_dim]*len(objects) # v_w when w is center


# count number of word occurences


In [5]:
with open('task1_objects_contexts_polish.txt', 'r') as f:
    pairs = [line.strip().split() for line in f if line.strip()]


objects = sorted(set([obj for obj, ctx in pairs]))
contexts = sorted(set([ctx for obj, ctx in pairs]))
vocab = sorted(set(objects + contexts))


word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for w, i in word2idx.items()}

# Hyperparameters
embedding_dim = 10
epochs = 10
learning_rate = 0.05
neg_samples = 5

# Initialize embeddings
U = np.random.randn(len(vocab), embedding_dim) * 0.01  # object vectors
V = np.random.randn(len(vocab), embedding_dim) * 0.01  # context vectors

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# build distribution for negative sampling
word_freq = Counter([ctx for obj, ctx in pairs])
freqs = np.array([word_freq.get(w, 1) for w in vocab])
neg_dist = freqs ** 0.75
neg_dist /= neg_dist.sum()   # normalize to 1

def get_negative_samples(pos_idx, k):
    negs = []
    while len(negs) < k:
        sample = np.random.choice(len(vocab), p=neg_dist)
        if sample != pos_idx:
            negs.append(sample)
    return negs

for epoch in tqdm(range(epochs), position=0):
    # np.random.shuffle(pairs)
    for obj, ctx in tqdm(pairs, position=1):
        obj_idx = word2idx[obj]
        ctx_idx = word2idx[ctx]
        neg_indices = get_negative_samples(ctx_idx, neg_samples)

        v_c = V[ctx_idx]
        u_o = U[obj_idx]
        u_negs = U[neg_indices]

        # Positive sample
        score_pos = np.dot(u_o, v_c)
        grad_pos = sigmoid(-score_pos)

        # Negative samples
        score_negs = np.dot(u_negs, v_c)
        grad_negs = sigmoid(score_negs)

        # Gradients
        grad_u_o = grad_pos * v_c
        grad_v_c = grad_pos * u_o - np.sum(grad_negs[:, None] * u_negs, axis=0)
        grad_u_negs = grad_negs[:, None] * v_c

        # Update
        U[obj_idx] -= learning_rate * grad_u_o
        V[ctx_idx] -= learning_rate * grad_v_c
        U[neg_indices] -= learning_rate * grad_u_negs

# Save object vectors
with open('object_vectors.txt', 'w') as f:
    for obj in objects:
        idx = word2idx[obj]
        vec = U[idx]
        f.write(f"{obj} {' '.join(map(str, vec))}\n")

  0%|          | 0/10 [01:12<?, ?it/s]


KeyboardInterrupt: 